### Planning with task decomposition

In this notebook, we will look at several methods of planning that allow you to decompose a complex task into smaller pieces.

Methods that we will cover include

1. Chain of Thought prompting (COT)
2. Tree of Thought prompting (TOT)
3. Plan and Solve

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") ## Put your OpenAI API key here
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY") ## Put your Tavily Search API key here
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_HUB_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_TRACING_V2"] = 'true' ## Set this as True
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com/' ## Set this as: https://api.smith.langchain.com/
os.environ["LANGCHAIN_HUB_API_URL"] = 'https://api.hub.langchain.com' ## Set this as : https://api.hub.langchain.com
os.environ["LANGCHAIN_PROJECT"] = 'llm-agents-planning'

## Chain of Thought Prompting

Learn More: https://deepgram.com/learn/chain-of-thought-prompting-guide

![COT](images/cot.jpeg)

In [3]:
open_source_model = 'mistralai/Mistral-7B-Instruct-v0.3'

In [18]:
## Add Together API Key
os.environ["TOGETHER_API_KEY"] = os.getenv("TOGETHER_API_KEY")
### import together
from together_llm import TogetherLLM
## Langchain imports
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [19]:
llm = TogetherLLM(model=open_source_model, temperature=0.3, max_tokens=800)

In [22]:
math_problem = ''' Solve the following problem step by step. First give the answer and then the steps.

A train leaves City A traveling towards City B at a speed of 120 km/h.
 Two hours later, a slower train leaves City B traveling towards City A at a speed of 80 km/h.
   If the distance between the two cities is 960 km, how far from City A will the two trains meet?
'''
# Create an LLMChain using the custom LLM
result = llm(math_problem)

mistralai/Mistral-7B-Instruct-v0.3 800 0.3


In [23]:
print(result)

 The two trains will meet 704 km away from City A.

Here's the step-by-step solution:

1. Let x be the distance the first train travels before the second train starts moving. Since the second train starts 2 hours later, the first train has been traveling for 2 hours when the second train starts.

2. The first train travels at a speed of 120 km/h, so in 2 hours, it covers a distance of 120 * 2 = 240 km. Therefore, x = 960 - 240 = 720 km.

3. Now, both trains are moving towards each other. Their combined speed is 120 + 80 = 200 km/h.

4. To find the time it takes for them to meet, we can use the formula: time = distance / combined speed. So, time = 720 / 200 = 3.6 hours.

5. Since the first train was already traveling for 2 hours when the second train started, the additional time it takes for them to meet is 3.6 - 2 = 1.6 hours.

6. During this additional 1.6 hours, the first train travels at a speed of 120 km/h. So, the distance it covers in this time is 120 * 1.6 = 192 km.

7. The fina

### Chain of Thought  with Self Consistency
- Run the problem several times and take the most common answer

![COT-SC](images/cot-sc.jpg)

### COT with Examples:

cot_prompt = '''Solve the following problem step-by-step, clearly showing your calculations and reasoning.
Problem: {math_problem}

Example of a math problem solved step by step

"Solve the following problem step-by-step, clearly showing your calculations and reasoning.
Problem: A group of friends went to a restaurant. The total bill was $120, and they wanted to leave a 15% tip. How much should each person pay if there were 5 friends splitting the bill and tip evenly?"

Response:

Step 1: Calculate the tip amount.

15% of the $120 bill is (15/100) * $120 = $18.
Step 2: Calculate the total cost including the tip.

The bill plus tip is $120 + $18 = $138.
Step 3: Divide the total cost by the number of friends.

Each friend should pay $138 / 5 friends = $27.60.
Therefore, each of the 5 friends should pay $27.60 to cover the bill and tip evenly..'''

## Plan and Solve

Paper: https://arxiv.org/pdf/2305.04091v3

![Plan_Solve](images/plan-solve.png)

In [32]:
plan_and_solve = ''' Solve the following problem by first devising a step by step plan for solving the problem. Then carry out the plan step by step

A train leaves City A traveling towards City B at a speed of 120 km/h.
 Two hours later, a slower train leaves City B traveling towards City A at a speed of 80 km/h.
   If the distance between the two cities is 960 km, how far from City A will the two trains meet?
'''
# Create an LLMChain using the custom LLM
result = llm(plan_and_solve)

mistralai/Mistral-7B-Instruct-v0.3 800 0.3


In [33]:
print(result)

 Step 1: Understand the problem
- Train A leaves City A at a speed of 120 km/h
- Train B leaves City B at a speed of 80 km/h (2 hours later)
- The distance between City A and City B is 960 km
- We need to find out how far Train A has traveled when it meets Train B.

Step 2: Calculate the combined speed of both trains
- Combined speed = Speed of Train A + Speed of Train B = 120 km/h + 80 km/h = 200 km/h

Step 3: Calculate the time it takes for both trains to meet
- Distance between the cities = 960 km
- Combined speed = 200 km/h
- Time = Distance / Combined speed = 960 km / 200 km/h = 4.8 hours

Step 4: Calculate the distance Train A travels in the first 2 hours
- Train A travels at a speed of 120 km/h for the first 2 hours = 120 km/h * 2 hours = 240 km

Step 5: Calculate the distance Train A travels in the remaining time (2.8 hours)
- Distance = Speed * Time = 120 km/h * 2.8 hours = 336 km

Step 6: Add the distances calculated in steps 4 and 5 to find the total distance Train A travels

## Tree of Thought

Read more about it here: https://www.promptingguide.ai/techniques/tot

![TOT](images/tot.png)

In [17]:
# set the LANGCHAIN_API_KEY environment variable (create key in settings)
from langchain import hub

tot_step1 = hub.pull("rachnogstyle/nlw_jan24_cot_step1")
tot_step2 = hub.pull("rachnogstyle/nlw_jan24_cot_step2")
tot_step3 = hub.pull("rachnogstyle/nlw_jan24_cot_step3")
tot_step4 = hub.pull("rachnogstyle/nlw_jan24_cot_step4")

print(tot_step1.template)
print(tot_step2.template)
print(tot_step3.template)
print(tot_step4.template)

Step1 :
 
I have a problem related to {input}. Could you brainstorm three distinct solutions? Please consider a variety of factors such as {perfect_factors}
A:
Step 2:

For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

{solutions}

A:
Step 3:

For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.

{review}

A:
Step 4:

Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution
{deepen_thought_process}

A:


In [36]:
model = "gpt-3.5-turbo"

chain1 = LLMChain(
    llm=ChatOpenAI(temperature=0.2, model=model),
    prompt=tot_step1,
    output_key="solutions"
)

chain2 = LLMChain(
    llm=ChatOpenAI(temperature=0.2, model=model),
    prompt=tot_step2,
    output_key="review"
)

chain3 = LLMChain(
    llm=ChatOpenAI(temperature=0.2, model=model),
    prompt=tot_step3,
    output_key="deepen_thought_process"
)

chain4 = LLMChain(
    llm=ChatOpenAI(temperature=0.2, model=model),
    prompt=tot_step4,
    output_key="ranked_solutions"
)

In [37]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

print(
        overall_chain(
            {
                "input": "human colonization of Mars",
                "perfect_factors": "The distance between Earth and Mars is very large, making regular resupply difficult"
            }
    )
)



> Entering new SequentialChain chain...

> Finished chain.
{'input': 'human colonization of Mars', 'perfect_factors': 'The distance between Earth and Mars is very large, making regular resupply difficult', 'ranked_solutions': 'Based on the evaluations and scenarios provided, the solutions can be ranked in the following order of promise:\n\n1. Developing sustainable and self-sufficient habitats on Mars\n2. Creating a system of supply missions with reusable spacecraft\n3. Establishing a network of resource extraction and processing facilities on Mars\n\nThe first solution of developing sustainable habitats on Mars is ranked highest due to its potential to address the long-term challenges of human habitation on the planet. By creating self-sufficient habitats, we can ensure the survival and well-being of future Martian colonists. The partnerships with private space companies and international space agencies offer a collaborative approach to leveraging expertise and resources for this am